# Generate Word Frequency in Audit Reports

## Steps Taken

1. Download the folder with audit reports from OneDrive
2. Iterate through the folder and get the folders with each country's reports
3. For each country's folder, get the country's reports
4. Lemmatize the words in each report
5. Get the frequency distribution of each text
6. Filter the words to remove noise
7. Enter the frequency distribution into a pandas dataframe
8. Merge each country's frequency distributions into one dataframe with the words as rows and report name as column
9. Enter each country as a spreadsheet in an excel workbook
10. Delete the downloaded documents

In [1]:
# link to audit_reports; publicly accessible
audit_reports_link = "https://stir-my.sharepoint.com/:f:/g/personal/fkc3_stir_ac_uk/Esgp-VMQyzBClY5vpTP9TsYBTCb16iA3NvelLEJM53VEgQ?e=7ejaR3"

### 2. Iterate through the folder and create a dictionary to match each country to files within it

In [17]:
import os
from pdfminer.high_level import extract_text
import nltk
from nltk.corpus import stopwords, words
import pandas as pd
from openai import OpenAI
import json
import functools

In [3]:
audit_reports_file_path = "./audit_reports"

In [4]:
country_audit_report_dict = {}

for root, dirs, files in os.walk(audit_reports_file_path):
    if (dirs): # ignore the country folders as children
        continue
    country_audit_report_dict[root] = sorted(files);

In [5]:
def save_reports_as_text():
    for country_folder_path, filenames in country_audit_report_dict.items():
        for filename in filenames:
            file_path = f'{country_folder_path}/{filename}'
            text_file_path = file_path.replace('.pdf', '.txt') # text path of pdf
            try:
                if file_path.endswith('.txt') or os.path.isfile(text_file_path): continue

                print(f'Doing {file_path}')               
                text = extract_text(f'{file_path}').lower()

                with open(text_file_path, 'w') as text_file:
                    text_file.write(text)
            
            except:
                print(f'{filename} has a problem')

#save_reports_as_text()

### 3. For each country's folder, get the country's reports as text files

In [6]:
country_texts_dict = {}

for country_folder_path, filenames in country_audit_report_dict.items():
    for filename in filenames:
        # print(f'Doing {country_folder_path}/{filename}')
        try:
            if not filename.endswith('.txt'): continue

            with open(f'{country_folder_path}/{filename}') as text:
                # print(filename, text.readline())
                if country_folder_path in country_texts_dict:
                    country_texts_dict[country_folder_path][filename] = text.read();
                    
                elif country_folder_path not in country_texts_dict:
                    country_texts_dict[country_folder_path] = {filename: text.read()}
                    
        except:
            print(f'{filename} has a problem')


### 4. Lemmatize the words in each report

In [7]:
lemmatizer = nltk.WordNetLemmatizer()

for country_path in country_texts_dict.keys():
    for report_name, report_text in country_texts_dict[country_path].items():
        if report_text:
            words_in_report = nltk.word_tokenize(report_text.lower()) 

            words_in_report = [lemmatizer.lemmatize(word) for word in words_in_report]

            country_texts_dict[country_path][report_name] = words_in_report

### 5. Get the frequency distribution of each text

In [8]:
country_report_freq_dist_dict = {}
stopwords_set = set(stopwords.words('english'))

# remove words with numbers and those that are stopwords
def clean_fdist(fdist):
    return {word: freq for word, freq in fdist.items() if word.isalpha() and word not in stopwords_set}

for country_path in country_texts_dict.keys():
    for report_name, report_words in country_texts_dict[country_path].items():
        fdist = nltk.FreqDist(report_words)
        fdist = clean_fdist(fdist)

        bigram_fdist = nltk.FreqDist(nltk.bigrams(report_words))
        # join the words if the words are all alphabets
        # and none of the words is a stopword
        bigram_fdist = {' '.join(key): value for key, value in bigram_fdist.items() if ''.join(key).isalpha() and set(key).isdisjoint(stopwords_set)}

        trigram_fdist = nltk.FreqDist(nltk.trigrams(report_words))
        trigram_fdist = {' '.join(key): value for key, value in trigram_fdist.items() if ''.join(key).isalpha() and set(key).isdisjoint(stopwords_set)}
        
        fdist.update(bigram_fdist)
        fdist.update(trigram_fdist)
        

        if country_path in country_report_freq_dist_dict:
            country_report_freq_dist_dict[country_path][report_name] = fdist
        else:
            country_report_freq_dist_dict[country_path] = {report_name: fdist}

### 7. Enter the frequency distributions into one dataframe per country

In [9]:
with pd.ExcelWriter('word-frequency.xlsx', engine='xlsxwriter') as writer:
    for country_path in country_report_freq_dist_dict.keys():
        df = pd.DataFrame()
        print(f'Writing {country_path} to file')
    
        for report_name, freq_dist in country_report_freq_dist_dict[country_path].items():
            if df.empty:
                df = pd.DataFrame.from_dict(dict(freq_dist), orient='index', columns=[report_name])
            else:
                other_df = pd.DataFrame.from_dict(dict(freq_dist), orient='index', columns=[report_name])
                df = pd.merge(df, other_df, 'outer', left_index=True, right_index=True)

        df.to_excel(writer, sheet_name=f'{country_path.removeprefix("./audit_reports/")}')
        print(f'Completed writing to {country_path}')

Writing ./audit_reports/Gambia to file
Completed writing to ./audit_reports/Gambia
Writing ./audit_reports/Zambia to file
Completed writing to ./audit_reports/Zambia
Writing ./audit_reports/South-Africa to file
Completed writing to ./audit_reports/South-Africa
Writing ./audit_reports/Nigeria to file
Completed writing to ./audit_reports/Nigeria
Writing ./audit_reports/Esthwani to file
Completed writing to ./audit_reports/Esthwani
Writing ./audit_reports/Kenya to file
Completed writing to ./audit_reports/Kenya
Writing ./audit_reports/Ghana to file
Completed writing to ./audit_reports/Ghana
Writing ./audit_reports/Malawi to file
Completed writing to ./audit_reports/Malawi
Writing ./audit_reports/Tanzania to file
Completed writing to ./audit_reports/Tanzania
Writing ./audit_reports/Uganda to file
Completed writing to ./audit_reports/Uganda


### 8. Filter financial words using AI

In [38]:
def get_api_key(secrets_file, key_name):
    # retrieves key from secrets_file if saved in the format
    #  key_name='example_api_key'
    with open(secrets_file) as env_file:
        for line in env_file.readlines():
            if line.startswith(key_name):
                return line.strip().removeprefix(f'{key_name}=')

openai_api_key = get_api_key('.env', 'OPENAI_DEV_KEY')

def send_prompt_to_GPT(instruction, background_context=None):
    
    client = OpenAI(api_key=openai_api_key)

    completion = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": background_context},
        {"role": "user", "content": instruction}
      ]
    )
    #print(completion.choices[0].message.content)
    return completion.choices[0].message.content

In [37]:
# handle the process of filtering the words
def filter_non_finance_words(excel_name):
    with pd.ExcelFile(excel_name) as excel_file:
        excel_sheetnames = get_sheet_names(excel_file)
        country_freq_dists_generator = gen_freq_dists(excel_sheetnames, excel_file)
        country_words_generator = gen_each_countrys_words(country_freq_dists_generator)
        country_finance_words_generator = send_each_countrys_words_to_GPT(country_words_generator)
        filtered_dataframe_generator = filter_finance_words_in_dataframe(country_freq_dists_generator, country_finance_words_generator)
        write_dataframes_to_excel('filtered-word-frequency.xlsx', filtered_dataframe_generator, excel_sheetnames)
            
        return next(filtered_dataframe_generator)

def get_sheet_names(excel_file):
    return excel_file.sheet_names

def gen_freq_dists(excel_sheetnames, excel_file):
    for name in excel_sheetnames:
        # yield the dataframe so they are not all saved to memory
        yield pd.read_excel(excel_file, name, index_col=0)

def gen_each_countrys_words(freq_dist_generator):
    for dataframe in freq_dist_generator:
        yield list(dataframe.index)

def send_each_countrys_words_to_GPT(country_words_generator):
    array_of_batch = None
    system_instruction = create_system_instruction()
    for country_words in country_words_generator:
        array_of_batch = split_words(country_words)

        # send each batch of strings to gpt
        # convert the result to json
        # reduce the arrays into one
        finance_array_of_batch = [json.loads(send_prompt_to_GPT(system_instruction, json.dumps(batch))) for batch in array_of_batch]
        yield functools.reduce(lambda x, y: x + y, finance_array_of_batch, [])

def split_words(country_words):
    max_limit = 500;
    if len(country_words) > max_limit:
        return [country_words[i:i + max_limit] for i in range(0, len(country_words), max_limit)]
    else:
        return [country_words]

def create_system_instruction():
    return ("You'll receive an list containing strings. "
            "Check each string for words related to finance, accounting or public administration. "
            "Some words are lower-case acronyms related to the topics previously mentioned "
            "such as ifmis, gifmis, ipsas, ifrs etc. "
            "If a string has at least one word matching the criteria, add it into an array."
            "For example, if given, ['ifrs', 'financial statement', 'walls painted'], "
            "return ['ifrs', 'financial statement']"
            "Basically, you are filtering out words that don't fit the criteria I've given you"
            "I expect only a valid JSON list back in the form of a string. Don't add markup.")

def filter_finance_words_in_dataframe(dataframe_generator, finance_words_generator):
    for dataframe, finance_words in zip(dataframe_generator, finance_words_generator):
    # for dataframe in dataframe_generator:
        yield dataframe[list(finance_words)]

def write_dataframe_to_excel(filename, dataframe_generator, sheet_names):
    with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
        for dataframe, sheet_name in zip(dataframe_generator, sheet_names):
            dataframe.to_excel(writer, sheet_name=sheet_name)
            print(f'Done writing {sheet_name} spreadsheet')

filter_non_finance_words('word-frequency.xlsx')

["ifrs", "financial statement"]
["accounting error", "financial audit", "public fund", "IFRS compliance", "budget review", "accountability report", "public administration", "accrual accounting", "treasury department", "financial regulation"]
["accounting", "accountant", "imprest", "reconciliation", "expense", "receipt", "investment", "payment", "property", "acquisition", "mortgage", "audit", "arrears", "compensation", "budgetary", "revenue", "expenditure", "excess", "impairment", "misappropriation", "allowance", "accrual", "ifrs", "ipsas", "gifmis"]
["ifrs", "financial statement", "acquittal sheet making", "acquittal sheet provided", "acquittal sheet revealed", "acquittal sheet two", "acquittal sheet wa", "acquittal sheet without", "acquittal showed", "acquittal without", "acquittal without supporting", "acquitted", "acquitted allowance", "acquitted authority", "acquitted k", "acquitted leaving", "acquitted making", "acquitted pay", "acquitted pay sheet", "acquitted payment", "acquitte

KeyboardInterrupt: 

In [27]:
ti = '["accounting", "financial statement"]'
t = json.loads(ti)
bi = ['a', 'b']

for a, b in zip(t, bi):
    print(a, b)

accounting a
financial statement b
